In [ ]:
'''this script takes a "history" frames reshapes it and merges it with pit stop

the actual data will be stored in 'raw_data' not in '2016race0'

the race_id (which is currently hard coded to 0) 
will be used with the ref_df dataframe to link it to race info (year,progressive number,track)

'''

In [53]:
#import tabula
import pandas as pd
import os
import numpy as np
import re
import datetime

In [54]:
# df_names = []
# for _pdf in os.listdir('2016race0'):
#     if (_pdf.lower().endswith(('.pdf'))) & ('jpg' not in _pdf.lower()):
#         _name = _pdf[7:-9]+'_df'
# #         print(_pdf)
# #         print(_name)
#         df_names.append(_name)
#         exec( _name+" = tabula.read_pdf('2016race0/'+_pdf,pages='all')")
# #        exec(_name+'["race_id"] = 0')
#         exec(_name+".to_csv('2016race0/'+_name+'.csv')")

In [55]:
# df_names = []
# for _csv in os.listdir('2016race0'):
#     if _csv.lower().endswith(('.csv')):
#         _name = _csv[:-4]
# #         print(_csv)
# #         print(_name)
#         df_names.append(_name)
#         exec(_name+" = pd.read_csv('2016race0/'+_name+'.csv')")
# #        exec(_name+"['race_id'] = 0")

In [56]:
#df = history_chart_2_df
df = pd.read_csv('2016race0/history_chart_2_df.csv')
pit_stop_df = pd.read_csv('2016race0/pit_stop_summary_6_df.csv')

In [57]:
df.rename(columns = {'NO':'NO.0','GAP':'GAP.0','TIME':'TIME.0'},inplace=True)

In [58]:
divisorz = df.loc[df['NO.0']=='NO'].index

In [59]:
# slicing the history dataframe horizontally
previous_divisor = -1
for _id,_val in enumerate(divisorz):
#     print (_id,_val)
    exec('df_'+str(_id)+'= df.loc[previous_divisor+1:_val-1]' )
    previous_divisor = _val
    div_counter = _id
last_id = div_counter + 1
exec('df_'+str(last_id)+'= df.loc[previous_divisor+1:]' )

In [60]:
starters = df_1['NO.0'].unique()

In [61]:
def split_PIT(x):
#     print(x)
    if x['NO'] not in starters:
        if pd.notna(x['NO']):
#        if x['NO'].notnull:
            if x['NO'][:-4] in starters:
#            return [1,2,3,4]
#                  print('correcting it')
                 return pd.Series([x['NO'][:-4],'PIT',x['TIME'],x['LAP']])
#             else:
#                  print('trashing it')
#                return pd.Series([None, None, None, None])
        else:
#             print('it is a nan')
            return pd.Series([None, None, None, None])
    else:
#         print('proper')
        return x
        #return [x['NO'],x['GAP'],x['TIME'],x['LAP']]

def split_TIMES(x):
    #print(x)
    if x['NO'] not in starters:
#         print('trashing it')
        return pd.Series([None, None, None, None])
    elif pd.isna(x['GAP']):
#         print('it is (hopefully) the leader')        
#         print(type(x),type(pd.Series([x['NO'],0,x['TIME'],x['LAP']])))
#         print(len(x),len(pd.Series([x['NO'],0,x['TIME'],x['LAP']])))
        return pd.Series([x['NO'],0,x['TIME'],x['LAP']])
    elif (' ' in str(x['GAP'])) & ('LAP' not in str(x['GAP'])):
#         print('correcting it (2)')
        timez = x['GAP'].split()
        return pd.Series([x['NO'],timez[0],timez[1],x['LAP']])
    elif pd.isna(x['TIME']):
#         print('it is a nan')        
        return pd.Series([x['NO'],0,x['GAP'],x['LAP']])
    else:
#         print('proper')
        return x

In [62]:
for j in range(len(divisorz)+1):
#     print('j is ', j)
    for i in [0,1,2,3,4]:
#         print('i is ', i)
#         print('doing lap',j*5+(i+1))
        if j*5+(i+1):
            # Take the horizontal slices and slice them vertically into laps 
            exec('df_lap_'+str(j*5+(i+1))+' = df_'+str(j)+'[["NO.'+str(i)+'","GAP.'+str(i)+'","TIME.'+str(i)+'"]]')
            # add lap feature 
            exec('df_lap_'+str(j*5+(i+1))+'["lap"] = '+str(j*5+(i+1)))
            # uniforming column names
            exec('df_lap_'+str(j*5+(i+1))+'.columns = ["NO","GAP","TIME","LAP"]')
            # putting 0 were 'GAP' is none (!!WORKAROUND!!the first row of a frame appear to behave differently than 
            # other rows resulting in adding new columns to the data in an incompatible way. 
            # considering a bug report to pandas developers
            exec('df_lap_'+str(j*5+(i+1))+'["GAP"].fillna(0,inplace=True)')
            # removing empty rows (where racing number is empty)
            exec('df_lap_'+str(j*5+(i+1))+' = df_lap_'+str(j*5+(i+1))+'[df_lap_'+str(j*5+(i+1))+'["NO"].notnull()]' )
            # solving the problem of 'PIT' value being collected in the 'NO' field
            exec('df_lap_'+str(j*5+(i+1))+' = df_lap_'+str(j*5+(i+1))+'.apply(split_PIT, axis =1)')
            # putting back the right names
            exec('df_lap_'+str(j*5+(i+1))+'.columns = ["NO","GAP","TIME","LAP"]')
            # removing empty rows (where racing number is empty).
            exec('df_lap_'+str(j*5+(i+1))+' = df_lap_'+str(j*5+(i+1))+'[df_lap_'+str(j*5+(i+1))+'["NO"].notnull()]' )            
            # solving the problem of 'NO' and 'GAP' fields merging
            exec('df_lap_'+str(j*5+(i+1))+' = df_lap_'+str(j*5+(i+1))+'.apply(split_TIMES, axis =1)')
            # putting back the right names
            exec('df_lap_'+str(j*5+(i+1))+'.columns = ["NO","GAP","TIME","LAP"]')
            # removing empty rows (where racing number is empty).
            exec('df_lap_'+str(j*5+(i+1))+' = df_lap_'+str(j*5+(i+1))+'[df_lap_'+str(j*5+(i+1))+'["NO"].notnull()]' )            

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/guido/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [63]:
laps_dfs=[]
for j in range(len(divisorz)+1):
    for i in [0,1,2,3,4]:
        exec('laps_dfs.append(df_lap_'+str(j*5+(i+1))+')')

In [64]:
race_df = pd.concat(laps_dfs,ignore_index=True)

race_df['race_id'] = 0

race_df.drop(race_df.index[race_df['NO'].isna()],axis=0, inplace=True)

race_df['NO'] = race_df['NO'].astype('int64')

race_df['LAP'] = race_df['LAP'].astype('int64')

#race_df['GAP'].fillna(0,inplace=True)

In [65]:
starters = starters.astype('int64')

finishers = race_df[race_df['LAP'] == max(race_df['LAP'])]['NO'].unique()


In [66]:
#pit_stop_df = pit_stop_summary_6_df

pit_stop_df = pit_stop_df[pit_stop_df['NO']!='NO']

pit_stop_df.rename(columns = {'S  T  OP':'STOP'},inplace=True)

pit_stop_df['NO'] = pit_stop_df['NO'].astype('int64')

pit_stop_df['LAP'] = pit_stop_df['LAP'].astype('int64')

In [71]:
new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')
new_df.drop(columns = ['Unnamed: 0','Unnamed: 6'], inplace=True)

In [87]:
new_df['TIMESECS'] = new_df['TIME'].apply(lambda x: int(x.split(':')[0])*60 + float(x.split(':')[1]))
new_df['TIME64'] = new_df['TIME'].apply(lambda x: datetime.datetime.strptime(x, "%M:%S.%f"))

In [68]:
# race_df.set_index(['NO','LAP'], inplace=True)
# pit_stop_df.set_index(['NO','LAP'], inplace=True)
# new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')



In [117]:
race_level_df = pit_stop_df[['STOP','NO']][pit_stop_df['NO'].isin(finishers)].groupby('NO').max().reset_index()
race_level_df['race_id'] = 0
#pit_stop_df[['STOP','NO']].groupby('NO').max()

In [93]:
ref_df = pd.read_csv('which_race.csv', header=0)

In [118]:
prova = ref_df.merge(race_level_df.reset_index(), on = ['race_id'], how ='inner')

In [119]:
prova

,race_id,year,race_in_year,track,index,NO,STOP
0,0,2016,0,Melbourne,0,3,3
1,0,2016,0,Melbourne,1,5,3
2,0,2016,0,Melbourne,2,6,2
3,0,2016,0,Melbourne,3,8,1
4,0,2016,0,Melbourne,4,11,2
5,0,2016,0,Melbourne,5,12,2
6,0,2016,0,Melbourne,6,19,2
7,0,2016,0,Melbourne,7,20,3
8,0,2016,0,Melbourne,8,22,3
9,0,2016,0,Melbourne,9,27,2


In [112]:
race_level_df = race_level_df.merge(ref_df.reset_index(), on = ['race_id'], how ='inner')

In [121]:
race_level_df = race_level_df.merge(ref_df, on = ['race_id'], how ='inner');
race_level_df

,NO,STOP,race_id,year,race_in_year,track
0,3,3,0,2016,0,Melbourne
1,5,3,0,2016,0,Melbourne
2,6,2,0,2016,0,Melbourne
3,8,1,0,2016,0,Melbourne
4,11,2,0,2016,0,Melbourne
5,12,2,0,2016,0,Melbourne
6,19,2,0,2016,0,Melbourne
7,20,3,0,2016,0,Melbourne
8,22,3,0,2016,0,Melbourne
9,27,2,0,2016,0,Melbourne


In [122]:
df = pd.read_csv("race_frames/history_chart_4_df.csv")

In [123]:
df

,Unnamed: 0,NO,GAP,TIME,NO.1,GAP.1,TIME.1,NO.2,GAP.2,TIME.2,NO.3,GAP.3,TIME.3,NO.4,GAP.4,TIME.4
0,0,44,NaN,1:38.829,44,NaN,2:01.377,44,NaN,1:59.340,44,NaN,1:23.397,44,NaN,1:23.045
1,1,6,1.132,1:39.961,6,0.672,2:00.917,6,0.655,1:59.323,6,1.037,1:23.779,6,1.432,1:23.440
2,2,33,1.922,1:40.751,33,1.286,2:00.741,33,1.127,1:59.181,33,1.708,1:23.978,33,2.060,1:23.397
3,3,27,2.432,1:41.261,27,2.266,2:01.211,27,1.601,1:58.675,27,2.456,1:24.252,27,3.232,1:23.821
4,4,7,5.352,1:44.181,7,3.759,1:59.784,7,2.293,1:57.874,7,3.348,1:24.452,7,4.411,1:24.108
5,5,3,PIT,1:45.827,19,4.564,1:58.510,19,2.931,1:57.707,19,4.554,1:25.020,19,5.837,1:24.328
6,6,19,7.431,1:46.260,5,5.185,1:58.193,5,3.492,1:57.647,5,5.328,1:25.233,5,6.656,1:24.373
7,7,5,8.369,1:47.198,77,6.044,1:57.724,77,3.891,1:57.187,77,6.432,1:25.938,77,7.885,1:24.498
8,8,77,9.697,1:48.526,11,7.154,1:57.965,11,4.450,1:56.636,11,6.967,1:25.914,11,8.594,1:24.672
9,9,11,10.566,1:49.395,55,8.648,1:57.402,55,4.904,1:55.596,55,7.498,1:25.991,55,9.457,1:25.004
